# Open and run analysis on multiple polygons <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **Compatability:** Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Products used:** 
[ga_ls8c_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3)

## Background
Many users need to run analyses on their own areas of interest. 
A common use case involves running the same analysis across multiple polygons in a vector file (e.g. ESRI Shapefile or GeoJSON). 
This notebook will demonstrate how to use a vector file and the Open Data Cube to extract satellite data from Digital Earth Australia corresponding to individual polygon geometries.

## Description
If we have a vector file containing multiple polygons, we can use the python package [geopandas](https://geopandas.org/) to open it as a `GeoDataFrame`. 
We can then iterate through each geometry and extract satellite data corresponding with the extent of each geometry. 
Further anlaysis can then be conducted on each resulting `xarray.Dataset`.

We can retrieve data for each polygon, perform an analysis like calculating NDVI and plot the data.

1. First we open the polygon using `geopandas`
2. Iterate through  a geodatframe, extracting satellite data from DEA's ODC
3. Calculate NDVI as an example analysis on one of the extracted satellite timeseries.
4. Plot NDVI for the polygon extent

***


## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Please note the use of `datacube.utils` package `geometry`: 
this is important for saving the coordinate reference system of the incoming shapefile in a format that the Digital Earth Australia query can understand.

In [2]:
%matplotlib inline

import datacube
import rasterio.crs
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datacube.utils import geometry

import sys
sys.path.append('../Scripts')
from dea_datahandling import load_ard
from dea_bandindices import calculate_indices
from dea_plotting import rgb, map_shapefile
from dea_temporaltools import time_buffer
from dea_spatialtools import xr_rasterize

### Connect to the datacube
Connect to the datacube database to enable loading Digital Earth Australia data.

In [3]:
# Temporary solution to account for Collection 3 data being in a different
# database on the NCI
try:
    dc = datacube.Datacube(app='Analyse_multiple_polygons', env='c3-samples')
except:
    dc = datacube.Datacube(app='Analyse_multiple_polygons')

## Analysis parameters

* `time_of_interest` : Enter a time, in units YYYY-MM-DD, around which to load satellite data e.g. `'2019-01-01'`
* `time_buff` : A buffer of a given duration (e.g. days) around the time_of_interest parameter, e.g. `'30 days'`
* `vector_file` : A path to a vector file (ESRI Shapefile or GeoJSON)
* `attribute_col` : A column in the vector file used to label the output `xarray` datasets containing satellite images. Each row of this column should have a unique identifier
* `products` : A list of product names to load from the datacube e.g. `['ga_ls7e_ard_3', 'ga_ls8c_ard_3']`
* `measurements` : A list of band names to load from the satellite product e.g. `['nbart_red', 'nbart_green']`
* `resolution` : The spatial resolution of the loaded satellite data e.g. for Landsat, this is `(-30, 30)`
* `output_crs` : The coordinate reference system/map projection to load data into, e.g. `'EPSG:3577'` to load data in the Albers Equal Area projection
* `align` : How to align the x, y coordinates respect to each pixel. Landsat Collection 3 should be centre aligned `align = (15, 15)` if data is loaded in its native UTM zone projection, e.g. `'EPSG:32756'` 

In [4]:
time_range = ('1986-01', '2020-10')

vector_file = '../Fitzgerald/FitzNP_PossVegPlots_2020.shp'
attribute_col = 'SiteName'

products = ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
measurements = ['nbart_red', 'nbart_green', 'nbart_blue', 'nbart_nir', 'nbart_swir_1', 'nbart_swir_2']
resolution = (-30, 30)
output_crs = 'EPSG:3577'
align = (0, 0)

### Look at the structure of the vector file
Import the file and take a look at how the file is structured so we understand what we are iterating through. 
There are two polygons in the file:

In [5]:
gdf = gpd.read_file(vector_file)
#gdf.head()

In [6]:
gdfp = gdf
gdfp['geometry'] = gdfp.geometry.buffer(2000)
#gdfp.head()

We can then plot the `geopandas.GeoDataFrame` using the function `map_shapefile` to make sure it covers the area of interest we are concerned with:

In [10]:
map_shapefile(gdfp, attribute=attribute_col)

Label(value='')

Map(center=[-33.92721378962945, 120.0132970707948], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### Create a datacube query object
We then create a dictionary that will contain the parameters that will be used to load data from the DEA data cube:

> **Note:** We do not include the usual `x` and `y` spatial query parameters here, as these will be taken directly from each of our vector polygon objects.

In [7]:
query = {'time': time_range,
         'measurements': measurements,
         'resolution': resolution,
         'output_crs': output_crs,
         'align': align,
         }

query

{'time': ('1986-01', '2020-10'),
 'measurements': ['nbart_red',
  'nbart_green',
  'nbart_blue',
  'nbart_nir',
  'nbart_swir_1',
  'nbart_swir_2'],
 'resolution': (-30, 30),
 'output_crs': 'EPSG:3577',
 'align': (0, 0)}

In [ ]:
# Dictionary to save results 
results = {}

# A progress indicator
i = 0

# Loop through polygons in geodataframe and extract satellite data
for index, row in gdf.iloc[:].iterrows():
    
    print(f'Feature: {i + 1}/{len(gdf)}')
    
    # Get the geometry
    try:
        # If on the DEA Sandbox
        geom = geometry.Geometry(row.geometry.__geo_interface__,
                                 geometry.CRS(gdf.crs))
    except:
        # If on the NCI
        crs_wkt = rasterio.crs.CRS(gdf.crs).to_wkt()
        geom = geometry.Geometry(row.geometry.__geo_interface__,
                                 geometry.CRS(crs_wkt))
    
    # Update dc query with geometry      
    query.update({'geopolygon': geom}) 
    
    # Load landsat
#    ds = load_ard(dc=dc, 
#                  products=products,
#                  min_gooddata=0.80,  # only take uncloudy scenes
#                  ls7_slc_off = True,                  
#                  group_by='solar_day',
#                  **query)

    # Generate a polygon mask to keep only data within the polygon
#    mask = xr_rasterize(gdf.iloc[[index]], ds) #for data vals
#    maskp = xr_rasterize(gdfp.iloc[[index]], ds) #for plot
    
    # Mask dataset to set pixels outside the polygon to `NaN`
#    ds = ds.where(mask)
#    dsp = ds.where(maskp) #for plot
    
    # Append results to a dictionary using the attribute
    # column as an key
#    results.update({str(row[attribute_col]) : ds})
    
#    for key in results.keys():
#        wname = key
       
    ## new bit
    for time, i in dsp.groupby('time'):
        # To plot the RGB array, first we need to load it
        # as a 3D numpy array with bands as the final axis
        rgb_array = np.transpose(i[["nbart_swir_1", "nbart_nir", "nbart_green"]]
                                 .to_array()
                                 .values, 
                                 axes=[1, 2, 0])
        # Divide by 3000 to keep values between 0 and 1,
        # and create a reasonably good colour stretch
        rgb_array = (rgb_array / 3000).clip(0, 1)
        
        fig, ax = plt.subplots(figsize=(10, 10))
        
        ax.imshow(rgb_array, 
                  extent=(i.x.min().item(), 
                          i.x.max().item(), 
                          i.y.min().item(), 
                          i.y.max().item()))
        # Export RGB image to file
        rgb_png = f'{wname}_{str(time)[0:10]}_{str(vector_file)[23:28]}_542.png'
        fig.savefig(rgb_png, bbox_inches='tight')
        plt.close(fig) 
    

Feature: 1/45
Finding datasets
    ga_ls5t_ard_3
    ga_ls7e_ard_3
    ga_ls8c_ard_3
Counting good quality pixels for each time step
Filtering to 532 out of 1606 time steps with at least 80.0% good quality pixels
Applying pixel quality/cloud mask
Loading 532 time steps
Feature: <xarray.Dataset>
Dimensions:       (x: 136, y: 136)
Coordinates:
    time          datetime64[ns] 2020-10-11T01:53:50.131347
  * y             (y) float64 -3.752e+06 -3.752e+06 ... -3.756e+06 -3.756e+06
  * x             (x) float64 -1.101e+06 -1.101e+06 ... -1.097e+06 -1.097e+06
    spatial_ref   int32 3577
Data variables:
    nbart_red     (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    nbart_green   (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    nbart_blue    (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    nbart_nir     (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    nbart_swir_1  (y, x) float32 nan nan nan nan nan nan ... nan nan na

## Loading satellite data

Here we will iterate through each row of the `geopandas.GeoDataFrame` and load satellite data.  The results will be appended to a dictionary object which we can later index to analyse each dataset.

In [9]:
# Dictionary to save results 
results = {}

# A progress indicator
i = 0

# Loop through polygons in geodataframe and extract satellite data
for index, row in gdf.iloc[:].iterrows():
    
    print(f'Feature: {i + 1}/{len(gdf)}')
    
    # Get the geometry
    try:
        # If on the DEA Sandbox
        geom = geometry.Geometry(row.geometry.__geo_interface__,
                                 geometry.CRS(gdf.crs))
    except:
        # If on the NCI
        crs_wkt = rasterio.crs.CRS(gdf.crs).to_wkt()
        geom = geometry.Geometry(row.geometry.__geo_interface__,
                                 geometry.CRS(crs_wkt))
    
    # Update dc query with geometry      
    query.update({'geopolygon': geom}) 
    
    # Load landsat
    ds = load_ard(dc=dc, 
                  products=products,
                  min_gooddata=0.80,  # only take uncloudy scenes
                  ls7_slc_off = True,                  
                  group_by='solar_day',
                  **query)

    # Generate a polygon mask to keep only data within the polygon
    mask = xr_rasterize(gdf.iloc[[index]], ds)
    
    # Mask dataset to set pixels outside the polygon to `NaN`
    ds = ds.where(mask)
    
    # Append results to a dictionary using the attribute
    # column as an key
    results.update({str(row[attribute_col]) : ds})
    
   
    # Update counter
    i += 1

Feature: 1/45
Finding datasets
    ga_ls5t_ard_3
    ga_ls7e_ard_3
    ga_ls8c_ard_3
Counting good quality pixels for each time step
Filtering to 532 out of 1606 time steps with at least 80.0% good quality pixels
Applying pixel quality/cloud mask
Loading 532 time steps
Feature: 2/45
Finding datasets
    ga_ls5t_ard_3
    ga_ls7e_ard_3
    ga_ls8c_ard_3
Counting good quality pixels for each time step
Filtering to 533 out of 1606 time steps with at least 80.0% good quality pixels
Applying pixel quality/cloud mask
Loading 533 time steps
Feature: 3/45
Finding datasets
    ga_ls5t_ard_3
    ga_ls7e_ard_3
    ga_ls8c_ard_3
Counting good quality pixels for each time step
Filtering to 531 out of 1607 time steps with at least 80.0% good quality pixels
Applying pixel quality/cloud mask
Loading 531 time steps
Feature: 4/45
Finding datasets
    ga_ls5t_ard_3
    ga_ls7e_ard_3
    ga_ls8c_ard_3
Counting good quality pixels for each time step
Filtering to 534 out of 1606 time steps with at least 80.

---
## Further analysis

Our `results` dictionary will contain `xarray` objects labelled by the unique `attribute column` values we specified in the `Analysis parameters` section:

In [10]:
# Define column names
colNames = ('date', 'site', 'red', 'green', 'blue', 'nir', 'swir1', 'swir2')

# Empty list
dfs = []

# Create a dataframe for each polygon
for key in results.keys():
    tempDF = pd.DataFrame(columns = colNames)
    tempDF['date'] = results[key].time.values
    tempDF['site'] = key
    tempDF['red'] = results[key].nbart_red.mean(dim=['x', 'y'], skipna = True)
    tempDF['green'] = results[key].nbart_green.mean(dim=['x', 'y'], skipna = True)
    tempDF['blue'] = results[key].nbart_blue.mean(dim=['x', 'y'], skipna = True)
    tempDF['nir'] = results[key].nbart_nir.mean(dim=['x', 'y'], skipna = True)
    tempDF['swir1'] = results[key].nbart_swir_1.mean(dim=['x', 'y'], skipna = True)
    tempDF['swir2'] = results[key].nbart_swir_2.mean(dim=['x', 'y'], skipna = True)
    
    #print(tempDF)
    #print('\n')

    # Try to append temporary DF to master DF
    dfs.append(tempDF)

masterDF = pd.concat(dfs, ignore_index=True)

In [11]:
# Preview data
print(masterDF)

                            date         site         red       green  \
0     1986-09-21 01:08:05.856567  EMB_001_035  442.305573  489.109039   
1     1987-09-08 01:14:24.005382  EMB_001_035  482.378815  543.001953   
2     1987-09-15 01:20:21.021825  EMB_001_035  546.152527  605.288696   
3     1987-10-01 01:20:42.284140  EMB_001_035  398.709045  443.899536   
4     1987-11-11 01:15:42.684642  EMB_001_035  428.064240  456.180084   
...                          ...          ...         ...         ...   
23805 2020-08-08 01:53:28.281356  EMB_045_065  355.473450  375.342407   
23806 2020-08-17 01:47:21.105146  EMB_045_065  485.758728  511.415588   
23807 2020-09-09 01:53:42.804248  EMB_045_065  490.191528  513.757324   
23808 2020-09-18 01:47:35.056912  EMB_045_065  592.776855  631.443237   
23809 2020-10-11 01:53:50.131347  EMB_045_065  507.717957  535.464783   

             blue          nir        swir1       swir2  
0      423.949493  1114.630615   976.508057  625.529785  
1      

Enter one of those values below to index our dictionary and conduct further analsyis on the satellite timeseries for that polygon.

In [12]:
masterDF.to_csv(f'Fitz_1986_202010_at80.csv')

In [ ]:
## zip everything up prior to download
# start up CLI and do an ls command to make sure processing folder exists (in this example it is SWWMP)
# run: tar -czvf 109084.tar.gz SWWMP to zip up work for 109084 - when unzipping can extract to same folder name

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** May 2020

**Compatible datacube version:** 

In [12]:
print(datacube.__version__)

1.7+142.g7f8581cf


## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)